In [1]:
# RUN THIS CELL EVERY RESET RUNTIME
WORKDIR='/content/AIC2022-VER'
DATADIR='/content/AIC2022-VER/data_sample' # replace this path
GITHUB_TOKEN=''

## Clone repo

In [ ]:
!pip install --upgrade --no-cache-dir gdown google-cloud-storage python-Levenshtein webcolors

In [9]:
REPO_URL=f'https://{GITHUB_TOKEN}@github.com/nhtlongcs/AIC2022-VER/'
!git clone $REPO_URL

Cloning into 'AIC2022-VER'...
remote: Enumerating objects: 2253, done.
remote: Counting objects: 100% (2253/2253), done.
remote: Compressing objects: 100% (1194/1194), done.
remote: Total 2253 (delta 1132), reused 2021 (delta 951), pack-reused 0
Receiving objects: 100% (2253/2253), 12.81 MiB | 3.79 MiB/s, done.
Resolving deltas: 100% (1132/1132), done.


In [ ]:
%cd $WORKDIR
!git checkout add-rfm
!git reset --hard HEAD
!git pull

In [11]:
!chmod +x $WORKDIR/tools/*

In [12]:
%%capture
%cd $WORKDIR
!pip install faiss-gpu
!pip install -r requirements.txt
!pip install -e . 

In [ ]:
%%capture
!python -m spacy download en_core_web_sm
!python -m nltk.downloader punkt wordnet omw-1.4

## Download checkpoints

In [2]:
import gdown 
def download(token, output: str):
    url = f'https://drive.google.com/uc?id={token}'
    gdown.download(url, output, quiet=False)

In [3]:
%cd $WORKDIR
download('1eylDX3tAJ2G0xhQsdOhU5uT1qH6SdzYK', 'retrieval-checkpoints.zip')
download('1VQ5vERytyTqDKlu2lM_ehk8BlgZ6wHgi', 'srl-checkpoints.zip')

/content/AIC2022-VER


Downloading...
From: https://drive.google.com/uc?id=1eylDX3tAJ2G0xhQsdOhU5uT1qH6SdzYK
To: /content/AIC2022-VER/retrieval-checkpoints.zip
100%|██████████| 10.0G/10.0G [01:44<00:00, 95.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VQ5vERytyTqDKlu2lM_ehk8BlgZ6wHgi
To: /content/AIC2022-VER/srl-checkpoints.zip
100%|██████████| 120M/120M [00:01<00:00, 82.3MB/s]


In [4]:
!unzip $WORKDIR/srl-checkpoints.zip
!unzip $WORKDIR/retrieval-checkpoints.zip
!rm -rf $WORKDIR/srl-checkpoints.zip
!rm -rf $WORKDIR/retrieval-checkpoints.zip

Archive:  /content/AIC2022-VER/srl-checkpoints.zip
   creating: artifacts/
   creating: artifacts/vehicle/
   creating: artifacts/color/
  inflating: artifacts/vehicle/model.ckpt  
  inflating: artifacts/color/model.ckpt  
Archive:  /content/AIC2022-VER/retrieval-checkpoints.zip
   creating: artifacts/model-2xjz0ux8:v25/
  inflating: artifacts/model-2xjz0ux8:v25/model.ckpt  
   creating: artifacts/model-1ityga7d:v20/
  inflating: artifacts/model-1ityga7d:v20/model.ckpt  
   creating: artifacts/model-3e6jnmrm:v16/
  inflating: artifacts/model-3e6jnmrm:v16/model.ckpt  
   creating: artifacts/model-55pfbgaf:v15/
  inflating: artifacts/model-55pfbgaf:v15/model.ckpt  
   creating: artifacts/model-3r4fsrlp:v29/
  inflating: artifacts/model-3r4fsrlp:v29/model.ckpt  
   creating: artifacts/model-3kto46es:v7/
  inflating: artifacts/model-3kto46es:v7/model.ckpt  
   creating: artifacts/model-5h72x53e:v18/
  inflating: artifacts/model-5h72x53e:v18/model.ckpt  


## Preprocess data

In [5]:
%%time 
%cd $WORKDIR
!./tools/preproc_motion.sh $DATADIR/meta/

/content/AIC2022-VER
Extracting motion map ...
3it [00:01,  2.16it/s]
6it [00:05,  1.04it/s]
DONE
CPU times: user 103 ms, sys: 29 ms, total: 132 ms
Wall time: 8.67 s


In [6]:
%%time 
%cd $WORKDIR
!./tools/preproc_srl.sh $DATADIR/meta

/content/AIC2022-VER
Augmenting data ...
100% 4/4 [00:00<00:00, 15.48it/s]
DONE
100% 2/2 [00:00<00:00, 27.57it/s]
DONE
Extracting SRL data ...
Load data_info successfully
downloading: 100%|##########| 387M/387M [00:09<00:00, 42.0MiB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 18.7kB/s]
Downloading: 100% 570/570 [00:00<00:00, 404kB/s]
Downloading: 100% 226k/226k [00:00<00:00, 683kB/s]
Downloading: 100% 455k/455k [00:00<00:00, 1.10MB/s]
Downloading: 100% 420M/420M [00:08<00:00, 49.4MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with 

In [7]:
%cd $WORKDIR
!./tools/preproc_relation.sh $DATADIR/ $DATADIR/meta/test_tracks.json

/content/AIC2022-VER
Extracting auxiliary tracks ...
  0% 0/36 [00:00<?, ?it/s]Number of auxiliary tracks: 9
100% 36/36 [00:03<00:00, 11.23it/s]
  0% 0/23 [00:00<?, ?it/s]Number of auxiliary tracks: 6
Number of auxiliary tracks: 4
100% 23/23 [00:00<00:00, 191.85it/s]
DONE
Generate neighbor mapping ...
100% 36/36 [00:00<00:00, 322.09it/s]
100% 23/23 [00:00<00:00, 568.15it/s]
DONE
Refine neighbor ...
  0% 0/2 [00:00<?, ?it/s]/content/AIC2022-VER/external/relation/relation_utils.py:65: RuntimeWarning: invalid value encountered in true_divide
  return np.dot(vect_a, vect_b)/(np.linalg.norm(vect_a)*np.linalg.norm(vect_b)) #default: L2 norm
100% 2/2 [00:00<00:00, 39.60it/s]
DONE


## Predict meta data

In [8]:
import os.path as osp

In [9]:
%cd $WORKDIR

track_json_path = osp.join(DATADIR,'meta/relation/neighbor_tracks.json')
!./tools/predict_srl.sh     configs/srl/col_infer.yml \
                            artifacts/color/model.ckpt \
                            configs/srl/veh_infer.yml \
                            artifacts/vehicle/model.ckpt \
                            $track_json_path \
                            $DATADIR/meta/extracted_frames \
                            ./results/classification/neighbors/

/content/AIC2022-VER
saving to folder ./results/classification/neighbors/
saving to ./results/classification/neighbors//color_prediction.json
saving to ./results/classification/neighbors//vehicle_prediction.json
Overriding configurating
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100% 20.4M/20.4M [00:00<00:00, 34.8MB/s]
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/AIC2022-VER/lightning_logs
P

In [10]:
%cd $WORKDIR

track_json_path = osp.join(DATADIR,'meta/test_tracks.json')
!./tools/predict_srl.sh     configs/srl/col_infer.yml \
                            artifacts/color/model.ckpt \
                            configs/srl/veh_infer.yml \
                            artifacts/vehicle/model.ckpt \
                            $track_json_path \
                            $DATADIR/meta/extracted_frames \
                            ./results/classification/test_tracks/

/content/AIC2022-VER
saving to folder ./results/classification/test_tracks/
saving to ./results/classification/test_tracks//color_prediction.json
saving to ./results/classification/test_tracks//vehicle_prediction.json
Overriding configurating
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100% 1/1 [00:00<00:00,  2.67it/s]
Saved file to ./results/classification/test_tracks//color_prediction.json
Overriding configurating
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
/usr/local/lib/python3.7/dis

In [11]:
%cd $WORKDIR
!python scripts/action/stop_turn_det.py -g $DATADIR/meta/test_tracks.json \
                                        -o results/classification/test_tracks/action_prediction.json

/content/AIC2022-VER
100% 2/2 [00:00<00:00, 269.63it/s]
Number of track which turns either left:  0
Number of track which turns either right:  0
Number of track which stops:  0
Number of straight-view track:  0


## Predict retrieval candidates

In [12]:
import os 
import torch
from opt import Opts
import os.path as osp 
from src.predictor import Predictor
from src.models import MODEL_REGISTRY
import gc
import json

In [13]:
config_data = {
    'model-55pfbgaf:v15': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'UTS-Best-0904'
    },
    'model-3kto46es:v7': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=224',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries_nlpaug_v2.jso')}n"
        ],
        'version': 'v2',
        'name': 'hcmussbj_eff3'
    },
    'model-3r4fsrlp:v29': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'UTS-Best-Second'
    },
    'model-5h72x53e:v18': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=256',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries_nlpaug_v2.json')}",
        ],
        'version': 'v2',
        'name': 'hcmussbj_eff3_best'
    },
    "model-2xjz0ux8:v25": {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=256',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries_nlpaug_v2.json')}",
        ],
        'version': 'v2',
        'name': 'hcmussbj_eff3'
    },
    'model-3e6jnmrm:v16': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'uts_eff3'
    },
    'model-1ityga7d:v20': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'uts_eff3'
    },
}

In [14]:
model_names = [
    'model-55pfbgaf:v15',
    'model-3kto46es:v7',
    'model-3r4fsrlp:v29',
    'model-5h72x53e:v18',
    "model-2xjz0ux8:v25",
    'model-3e6jnmrm:v16',
    'model-1ityga7d:v20',
]
data_params = [ f"data.track.args.json_path={osp.join(DATADIR,'meta/test_tracks.json')}",
                f"data.track.args.image_dir={osp.join(DATADIR,'meta/extracted_frames')}",
                f"data.text.args.shuffle=False",
                f"data.track.args.motion_path={osp.join(DATADIR,'meta/motion_map')}"
            ] 

In [15]:
def run_predict(cfg, model, mode, bs, savedir, top_k=184, NUM_TTA=1):
    NUM_TTA = max(NUM_TTA, 1)
    for tta_id in range(NUM_TTA):
        savedir_tta = osp.join(savedir, str(tta_id))
        os.makedirs(savedir_tta, exist_ok=True)
        infer = Predictor(cfg=cfg, model=model, mode=mode, batch_size=bs, savedir=savedir_tta, top_k=top_k) # TOTAL NUMBER OF TRACKS (CANNOT SET -1)
        infer.predict()
        del infer
        gc.collect()
        torch.cuda.empty_cache()

In [16]:
%cd $WORKDIR

for i, model_name in enumerate(model_names):
    print(f'\n === running {model_name} {i+1}/{len(model_names)}=== \n')
    params = ['-o'] + config_data[model_name]['params'] + data_params
    savedir = osp.join(WORKDIR, f"results/retrieval/{model_name}")
    resume_ckpt = osp.join(WORKDIR, f"artifacts/{model_name}/model.ckpt")
    cfg_path = osp.join(WORKDIR,"configs/inference.yml") if config_data[model_name]['version'] == 'v1' else osp.join(WORKDIR,"configs/inferencev2.yml")
    cfg = Opts(cfg=cfg_path).parse_args(params)
    model = MODEL_REGISTRY.get(cfg.model["name"])(cfg)
    model = model.load_from_checkpoint(resume_ckpt, config=cfg, strict=False)
    run_predict(cfg, model, "complex", 32, savedir, top_k=184)


 === running model-55pfbgaf:v15 1/7=== 

Overriding configurating


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Using CPU to retrieve


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Saved query_embeds.json to /content/AIC2022-VER/results/retrieval/model-1ityga7d:v20/0
Saved track_embeds.json to /content/AIC2022-VER/results/retrieval/model-1ityga7d:v20/0
Save query results to  /content/AIC2022-VER/results/retrieval/model-1ityga7d:v20/0/retrieval_results.json


In [18]:
save_dir = osp.join(WORKDIR, 'results/retrieval')
results = {}
for m in model_names:
    for idx in range(1): # TTA_IDS = [0]
      pred_json = osp.join(save_dir, m, str(idx), 'retrieval_results.json')      
      with open(pred_json, 'r') as f:
          data = json.load(f)
      for q, preds in data.items():
          if not(q in results.keys()):
              results[q] = {}
              results[q] = {}
          tmp = {}
          for i, k in enumerate(data[q]['pred_ids']):
              tmp[k] = data[q]['scores'][i]
          results[q][m] = tmp

# print(results.keys())
# print(data['1ed5b63a-0840-4fc3-8150-dd73b9b809ce']['pred_ids'])

q_ids = results.keys()
results_e = {}
for q in q_ids:
    results_e[q] = {}
    for model_name, model_res in results[q].items():
        for q_i in model_res.keys():
            if q_i in results_e[q]:
                results_e[q][q_i].append(1 * model_res[q_i])
            else:
                results_e[q][q_i] = [1 * model_res[q_i]]
# print(results_e['1ed5b63a-0840-4fc3-8150-dd73b9b809ce'])

for q in q_ids:
    for k in results_e[q]:
        results_e[q][k] = sum(results_e[q][k])
# print(results_e['1ed5b63a-0840-4fc3-8150-dd73b9b809ce'])

result_final = {}
for q in q_ids:
    result_final[q] = [(k, v) for k, v in results_e[q].items()]
    result_final[q] = sorted(result_final[q], key=lambda tup: tup[1], reverse=True)
    result_final[q] = [x[0] for x in result_final[q]]

# print(result_final['1ed5b63a-0840-4fc3-8150-dd73b9b809ce'])
result_path = osp.join(save_dir,'ensemble.json')
with open(result_path, 'w') as f:
    json.dump(result_final, f, indent=4)
print(f'saved prediction file at: {result_path}')

saved prediction file at: /content/AIC2022-VER/results/retrieval/ensemble.json


### Sanity check (only for real test data, have 184 queries)

In [19]:
import json
def load(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data
sub = load(result_path)
print(len(sub.keys()))
for k in sub.keys():
  if len(set(sub[k])) != 184:
      print(len(set(sub[k])))
print(sub[k])

2
2
2
['c19db37b-ec12-43ca-902e-5ff2d8ef3b5a', 'a1186b5d-6be7-464a-aa79-bad9acdf2895']


## Refine results

In [29]:
!./tools/refine.sh  $WORKDIR/results/retrieval/ensemble.json \
                    $DATADIR/meta/srl/postproc/test_srl.csv \
                    $WORKDIR/results/test_relation_action_f1 \
                    $DATADIR/meta/relation/test_tracks_relation.json \
                    $DATADIR/meta/relation/neighbor_tracks.json \
                    $WORKDIR/results/classification/ \
                    $WORKDIR/results/ 

parse relation result and save to /content/AIC2022-VER/results/test_relation_action_f1
100% 2/2 [00:00<00:00, 912.60it/s]
run refinement, save result to /content/AIC2022-VER/results/
100% 2/2 [00:00<00:00, 1069.43it/s]
n svo: 0
saved full class submission to: /content/AIC2022-VER/results/ensemble_full_Class.json
saved action only submission to: /content/AIC2022-VER/results/ensemble_action.json
saved vehcol only submission to: /content/AIC2022-VER/results/ensemble_vehcol.json
